In [14]:
import pdfplumber
from fuzzywuzzy import fuzz
import time
import csv
import tqdm as notebook_tqdm

In [2]:
import os
import json
import requests
import time
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = "https://ai-bcds.openai.azure.com/",
  api_key= "8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23",
  api_version="2024-05-01-preview"
)

assistant = client.beta.assistants.create(
  model="gpt-4o-mini", # replace with model deployment name.
  instructions="Tu és um perito que trabalha na Fidelidade em investimentos a prazo. Toda a informação que dás é com base nos ficheiros disponibilizados.",
  tools=[{"type":"file_search"}],
  tool_resources={"file_search":{"vector_store_ids":["vs_ftCso8RzHlBSbUwcqVWP3RiG"]},"code_interpreter":{"file_ids":["assistant-WuJZaGApRBgPZGvfPJwTBF","assistant-WQbe2oYNiunshZFamDkAMe","assistant-WA9mo2ooGuCakdtLYe7S5Y","assistant-VmGKDJtKWW8mLr7RsNDHJ4","assistant-VcoN7vmxYPZCBfSTaFsEQ3","assistant-TaHujXgSvATXXH1V5gBTDN","assistant-NMXvUzELtRbwtQMoTXeApo","assistant-LvBZjKCKvtwUvCCTgpfWim","assistant-GWrPgzCPp8si1AH2yKKJBB","assistant-F5nrFbNEjGxEqeSCmkdLAg","assistant-EwB2EpAtrmQVUrtxvg79HB","assistant-BFYazbjKgte4W5u9Jgrs6b","assistant-96mBrzvMJbaTjLYcCEG3S2","assistant-8phYrqvFLjksiThSKqSgkq","assistant-7GHqFyWe29skpMH8Ug4QoZ","assistant-6zJbn35vHos3RRzqJUiVMQ","assistant-5dMu4U3q2VUkjhP4LvhAGb","assistant-4hduKqwh6y7AWcUohbxMC3","assistant-426n1B453C2wqH2AnDc2mh","assistant-3UFjejDd1iMmYhBuj2bgi8"]}},
  temperature=0.44,
  top_p=1
)

In [3]:
def extract_qa_pairs_from_pdf(pdf_path):
    def save_current_pair():
        if question_lines and answer_lines:
            qa_pairs.append((
                " ".join(question_lines).strip(),
                " ".join(answer_lines).strip()
            ))

    qa_pairs = []
    question_lines, answer_lines = [], []
    mode = None  # 'question' or 'answer'

    with pdfplumber.open(pdf_path) as pdf:
        full_text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())

    for line in full_text.splitlines():
        line = line.strip()
        if not line:
            continue

        if line.startswith("•"):
            save_current_pair()
            question_lines = [line.removeprefix("•").strip()]
            answer_lines = []
            mode = 'question'

        elif line.startswith("R:"):
            answer_lines = [line.removeprefix("R:").strip()]
            mode = 'answer'

        elif mode == 'question':
            question_lines.append(line)
        elif mode == 'answer':
            answer_lines.append(line)

    save_current_pair()  # Save any remaining pair
    return qa_pairs

In [4]:
qa_pairs = extract_qa_pairs_from_pdf('./docs/Documents for training and evaluation-20250507\Questions_Answers_Censored.pdf')

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [10]:
# Ask a question to your assistant and return the answer
def query_assistant(question):
    thread = client.beta.threads.create()
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=question)

    # Run the thread
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
        )
    # Wait until the run completes
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status in ["completed", "failed"]:
            break
        time.sleep(1)

    if run.status == "failed":
        return "[ERROR] Assistant run failed."

    # Retrieve messages (latest one from assistant)
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for message in messages.data:
        if message.role == "assistant":
            return message.content[0].text.value.strip()
    return "[No assistant response]"


# --- Evaluate and Save Results ---
def evaluate_and_save(pdf_path, output_csv="results.csv"):
    qa_pairs = extract_qa_pairs_from_pdf(pdf_path)
    total = len(qa_pairs)
    correct = 0
    total_time = 0.0

    results = []

    for idx, (question, expected) in enumerate(qa_pairs, 1):
        print(f"\n--- Q{idx} ---")
        print("Question:", question)

        start = time.time()
        answer = query_assistant(question)
        response_time = time.time() - start
        total_time += response_time

        similarity = fuzz.ratio(answer.lower(), expected.lower())
        match = "Yes" if similarity >= 70 else "No"
        if match == "Yes":
            correct += 1

        print("Answer:", answer)
        print("Expected:", expected)
        print(f"Similarity: {similarity}, Match: {match}, Time: {response_time:.2f}s")

        results.append({
            "Question": question,
            "Expected Answer": expected,
            "Chatbot Answer": answer,
            "Similarity": similarity,
            "Match": match,
            "Response Time (s)": round(response_time, 2)
        })

    accuracy = (correct / total * 100) if total else 0
    avg_response = (total_time / total) if total else 0

    # Save to CSV
    with open(output_csv, "w", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=results[0].keys())
        writer.writeheader()
        writer.writerows(results)

    print("\n===== Summary =====")
    print(f"Total: {total}")
    print(f"Correct: {correct}")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Average Response Time: {avg_response:.2f}s")
    print(f"Results saved to: {output_csv}")


evaluate_and_save('./docs/Documents for training and evaluation-20250507\Questions_Answers_Censored.pdf')

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox



--- Q1 ---
Question: O Fidelidade Savings é mais indicado para que perfil de risco?
Answer: O produto **Fidelidade Savings** é mais indicado para diferentes perfis de risco, dependendo da opção escolhida. Aqui estão as opções disponíveis e os perfis de risco correspondentes:

1. **Opção Seguro**: Destina-se a clientes com perfil **prudente**, que valorizam a garantia de 100% do capital investido e um rendimento garantido (variável). É a opção mais conservadora, ideal para quem busca segurança e menor risco【44:3†source】【44:9†source】.

2. **Opção Proteção**: Esta opção é adequada para clientes com perfil **equilibrado**. Oferece uma garantia de 90% do capital investido, permitindo um maior potencial de ganho do que um rendimento garantido, mas com um risco diversificado em ações, obrigações e mercadorias【44:8†source】【44:12†source】.

3. **Opção Dinâmico**: Destinada a clientes com perfil **dinâmico**. Esta opção não oferece garantia do capital investido e é voltada para aqueles que aceit

| **Metric**            | **What It Measures**                                                                 |
|-----------------------|--------------------------------------------------------------------------------------|
| **Accuracy**          | Percentage of chatbot answers that closely match the expected answer.               |
| **Similarity Score**  | Fuzzy match score (0–100) comparing chatbot’s answer to the expected answer.        |
| **Match (Yes/No)**    | Whether the answer is considered correct (based on a similarity threshold, e.g. 70).|
| **Response Time**     | Time (in seconds) taken by the chatbot to respond to a question.                    |
| **Average Response**  | Average of all response times — reflects chatbot’s overall speed/efficiency.       |


# Problem: its string similarity so it checks how close the respective tokens are.

In [ ]:
from sentence_transformers import SentenceTransformer, util
import time
import csv

# Load model once globally for efficiency
model = SentenceTransformer('all-MiniLM-L6-v2')

def evaluate_and_save(pdf_path, output_csv="results.csv"):
    qa_pairs = extract_qa_pairs_from_pdf(pdf_path)
    total = len(qa_pairs)
    correct = 0
    total_time = 0.0

    results = []

    for idx, (question, expected) in enumerate(qa_pairs, 1):
        print(f"\n--- Q{idx} ---")
        print("Question:", question)

        start = time.time()
        answer = query_assistant(question)
        response_time = time.time() - start
        total_time += response_time

        # Encode both texts to embeddings
        emb_answer = model.encode(answer, convert_to_tensor=True)
        emb_expected = model.encode(expected, convert_to_tensor=True)

        # Calculate cosine similarity (value between 0 and 1)
        cosine_sim = util.pytorch_cos_sim(emb_answer, emb_expected).item()

        # Threshold for a "match" - adjust as needed (e.g., 0.7)
        match = "Yes" if cosine_sim >= 0.7 else "No"
        if match == "Yes":
            correct += 1

        print("Answer:", answer)
        print("Expected:", expected)
        print()
        print('----------------------------------')
        print(f"Cosine Similarity: {cosine_sim:.2f}, Match: {match}, Time: {response_time:.2f}s")

        results.append({
            "Question": question,
            "Expected Answer": expected,
            "Chatbot Answer": answer,
            "Cosine Similarity": round(cosine_sim, 2),
            "Match": match,
            "Response Time (s)": round(response_time, 2)
        })

    accuracy = (correct / total * 100) if total else 0
    avg_response = (total_time / total) if total else 0

    # Save results to CSV
    with open(output_csv, "w", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=results[0].keys())
        writer.writeheader()
        writer.writerows(results)

    print("\n===== Summary =====")
    print(f"Total: {total}")
    print(f"Correct: {correct}")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Average Response Time: {avg_response:.2f}s")
    print(f"Results saved to: {output_csv}")

# Then call this with your PDF path:
evaluate_and_save('./docs/Documents for training and evaluation-20250507/Questions_Answers_Censored.pdf')


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox



--- Q1 ---
Question: O Fidelidade Savings é mais indicado para que perfil de risco?
Answer: O produto **Fidelidade Savings** é mais indicado para diferentes perfis de risco, dependendo da opção escolhida:

1. **Opção Seguro**: Destina-se a clientes com perfil **prudente**, que valorizam a garantia de 100% do capital investido e um rendimento garantido (variável) ao longo da vigência do contrato. É a opção mais conservadora e é apropriada para aqueles que buscam segurança e estabilidade em seus investimentos【44:3†source】.

2. **Opção Proteção**: Adequada para clientes com perfil **equilibrado**, que desejam um investimento com maior potencial de valorização do que um rendimento garantido, mas com uma proteção de 90% do capital investido. Esta opção é para aqueles que aceitam um nível moderado de risco, buscando um equilíbrio entre segurança e rentabilidade【44:9†source】【44:12†source】.

3. **Opção Dinâmico**: Voltada para clientes com perfil **dinâmico**, que estão dispostos a assumir ri

In [ ]:
# Assistant failed in some questions; testei a 9 no streamlit e funcionou
# Podemos ajustar a treshhold para aquilo que assumimos correto

## Summary of the Evaluation Script

This script evaluates the quality of answers generated by an assistant to a set of questions extracted from a PDF file. Instead of relying on simple string matching, it uses **semantic similarity** to compare the assistant's answers to the expected answers.

### Key Steps:
- Extracts question-answer pairs from a PDF.
- Sends each question to the assistant and retrieves its answer.
- Converts both the assistant's answer and the expected answer into semantic embeddings using a pretrained Sentence Transformer model.
- Computes the cosine similarity between the embeddings to measure how close the meanings are.
- Determines if the assistant's answer matches the expected answer based on a similarity threshold.
- Records results, including similarity scores, matches, and response times.
- Saves detailed evaluation results to a CSV file and prints a summary report.

### Benefits:
- Captures meaning rather than exact wording.
- Handles paraphrased or reworded answers effectively.
- Provides a more robust and meaningful assessment of the assistant’s performance.
